#Libraries and Data

In [57]:
#import libraries
import numpy as np
import pandas as pd
import itertools
from prophet import Prophet
import yfinance
import pickle
from statsmodels.tsa.stattools import adfuller

import xgboost as xgb
from datetime import datetime

from dataProcessing import DataProcessing
# Remove Future Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from ta import add_all_ta_features




from sklearn.model_selection import train_test_split

# Machine Learning
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

# Evaluation
from sklearn.metrics import precision_score

# Reporting
import matplotlib.pyplot as plt

In [44]:
def load_and_setup_data(sybmol,input_data):
    df = pd.read_csv("../stock_historical_data/{}.csv".format(sybmol))
    df.set_index("Date", inplace=True)
    new_data = pd.DataFrame(input_data)
    new_data.set_index("Date", inplace=True)
    concatenated_df = pd.concat([df,new_data])
    return df

def fetch_stock_data(tickers):
    data = yfinance.download(tickers,period='1d')
    #print(data)
    return data
    

In [46]:
data = load_and_setup_data("RELI",[])
loaded_model_prophet1= {}
with open('../TrainedModel/xg/{}_model_phrophet.pkl'.format(symbol), 'rb') as f:
     loaded_model_prophet1 = pickle.load(f)
print(data)

                  Open        High        Low      Close  Volume
Date                                                            
2021-02-09   81.000000  136.050003  78.000000  87.300003  136673
2021-02-10  105.000000  105.000000  90.000000  97.349998   74767
2021-02-11  102.000000  102.000000  90.000000  91.050003   18300
2021-02-12   89.550003   89.699997  72.150002  78.150002   32280
2021-02-16   78.750000   81.000000  75.150002  75.900002   17207
...                ...         ...        ...        ...     ...
2023-07-10    4.720000    4.920000   4.710000   4.790000   20600
2023-07-11    4.740000    4.840000   4.680000   4.720000   26200
2023-07-12    4.800000    4.800000   4.430000   4.430000   39800
2023-07-13    4.620000    4.650000   4.400000   4.490000    8600
2023-07-14    4.490000    4.660000   4.115800   4.200000   21853

[611 rows x 5 columns]


In [115]:
#symbols = ["OLECTRA.NS","LT.NS","CONCOR.NS","ELGIEQUIP.NS","IOC.NS","BEL.NS","TATAELXSI.NS","^NSEI"]
symbols = ["OLECTRA.NS","CONCOR.NS","ELGIEQUIP.NS","IOC.NS","BEL.NS","TATAELXSI.NS","^NSEI","RELI","HDFCBANK.NS","TATAMOTORS.NS","SBIN.NS","TCS.NS","TITAN.NS","SUNPHARMA.BO","TECHM.NS", "ASIANPAINT.NS","TATACONSUM.NS"]
#symbols = ["RELI","HDFCBANK.NS"]
Icdate = 0
Iresult = 1
IisClassification = 2
Imodel = 3
Istock = 4
Iclose = 5
Iprob = 6
  # Add the tickers you want to fetch data for
loaded_models = {}
result = []

final_result = {}
final_target = {}
stock_data = fetch_stock_data(symbols)
print(stock_data)
finaldata = {}
current_date = datetime.now().strftime("%d-%m-%Y")

for symbol in symbols:
    stock_name = symbol
    if symbol == "^NSEI":
        stock_name = "NSEI"
    print(symbol)
    input_data = {
                    # "Open" : stock_data.head(1).Open[ticker].values[0],
                    "Open" : [stock_data.head(1).Open[symbol].values[0]],
                    "Close" : [stock_data.head(1).Close[symbol].values[0]],
                    "High" : [stock_data.head(1).High[symbol].values[0]],
                    "Low" : [stock_data.head(1).Low[symbol].values[0]], 
                    "Volume" : [stock_data.head(1).Volume[symbol].values[0]],
                    "Date":[np.datetime_as_string(stock_data.index, unit='D')[0]]
    }    #print(input_data)
    data = load_and_setup_data(symbol,input_data)
    # data.to_csv(f"../stock_historical_data/{symbol}.csv")
    #data = DataProcessing(symbol,data)
    # Specify Target
    # data.df.loc[data.df["Range"].shift(-1) > data.df["Avg_Range"], "TARGET"] = 1
    # data.df.loc[data.df["Range"].shift(-1) <= data.df["Avg_Range"], "TARGET"] = 0
    loaded_model = {}
    loaded_model_prophet = {}

    #feature_item = {}
    with open('../TrainedModel/xg/{}_model_phrophet.pkl'.format(symbol), 'rb') as f:
        loaded_model_prophet = pickle.load(f)
    with open('../TrainedModel/xg/{}_model_2.pkl'.format(symbol), 'rb') as f:
        loaded_model = pickle.load(f)
   # with open('../TrainedModel/xg/{}_features.txt'.format(symbol), 'rb') as f:
    #    feature_item = pickle.load(f)
    #final training
    loaded_models[symbol] = loaded_model
    data  = add_all_ta_features(data, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)
    data = data.dropna()
    data["ds"] = data.index
   # data["Date"] =  pd.to_datetime(data['Date'])
    data["y"] = data["Close"]


    forecast = loaded_model_prophet.predict(data)
    data = data.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
    
    #data = data.drop("index",axis=1)
    
    #get some variables
    prophet_variables = forecast.loc[:, ["trend", "weekly",  "multiplicative_terms","trend_upper","trend_lower"]]
    df_xgb = pd.concat([data, prophet_variables], axis = 1)
    non_stationaries = []
    for col in df_xgb.columns:
        if col != "multiplicative_terms" and col != "Date" and col != "ds" :
            #print(col)
            dftest = adfuller(df_xgb[col].values)
            p_value = dftest[1]
            t_test = dftest[0] < dftest[4]["1%"]
            if p_value > 0.05 or not t_test:
                non_stationaries.append(col)
    #print(f"Non-Stationary Features Found: {len(non_stationaries)}")
    
    df_stationary = df_xgb.copy()

    df_stationary = df_stationary.drop(columns=["Date","ds"])
    df_stationary[non_stationaries] = df_stationary[non_stationaries].pct_change()
    print(df_stationary)
    df_stationary = df_stationary[loaded_model.feature_names_in_]

   # score = loaded_model.predict(df_stationary)
    y_train_pred = loaded_model.predict_proba(df_stationary)[:,1]
    
    #print(y_train_pred)
        #df_xgb.head(1)
    #print(forecast)
   # final_target[symbol] = data.df["TARGET"]
   # df = data.df[feature_item]
   # finaldata[symbol]= df 
    stock_name_only = stock_name.replace(".NS","")
    train_yhat = loaded_model.predict(df_stationary[-1:])
    train_yhat_proba = loaded_model.predict_proba(df_stationary[-1:])
    greater = 0
    one = train_yhat_proba[0][0]
    two = train_yhat_proba[0][1]
    if one > two:
        greater = one*100
    else:
        greater = one*100
   # print(greater)
    #final_result[symbol] = [current_date,train_yhat[0],1,'xg',stock_name_only,stock_data.head(1).Close[symbol].values[0],round(greater,2)]
    result.append([current_date,train_yhat[0],1,'xg',stock_name_only,data.iloc[-1].Close,round(greater,2)])

print(result)
    

[*********************100%***********************]  17 of 17 completed
               Adj Close                                                   \
           ASIANPAINT.NS      BEL.NS   CONCOR.NS ELGIEQUIP.NS HDFCBANK.NS   
Date                                                                        
2023-07-14       3429.75  127.150002  688.799988   572.950012      1644.5   

                                                                   ... Volume  \
               IOC.NS   OLECTRA.NS  RELI     SBIN.NS SUNPHARMA.BO  ...   RELI   
Date                                                               ...          
2023-07-14  96.599998  1373.650024  4.35  584.400024  1072.400024  ...  42000   

                                                                            \
             SBIN.NS SUNPHARMA.BO TATACONSUM.NS TATAELXSI.NS TATAMOTORS.NS   
Date                                                                         
2023-07-14  12190152        33392       1705168       315095 

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


              Open      High       Low     Close    Volume  volume_adi  \
0              NaN       NaN       NaN       NaN       NaN         NaN   
1    -8.010024e-08  0.064935  0.027397  0.080000  2.045161         inf   
2     8.000006e-02  0.121951 -0.060000 -0.037037  0.066737   -0.451502   
3    -9.876539e-02 -0.130435  0.028369 -0.006410 -0.630586    0.335275   
4     1.301368e-01  0.031250  0.075862  0.032258 -0.426075   -0.048036   
...            ...       ...       ...       ...       ...         ...   
4573  0.000000e+00  0.000000  0.000000 -0.015639  0.057912    0.126968   
4574  6.194017e-02  0.032417 -0.000923 -0.022630 -0.370363   -0.028506   
4575 -2.391457e-02 -0.048464  0.012317  0.008110 -0.415965    0.035545   
4576  0.000000e+00  0.000000  0.000000  0.000000  0.001617    0.034381   
4577  0.000000e+00  0.000000  0.000000  0.000000  0.000000    0.033238   

      volume_obv  volume_cmf  volume_fi     volume_em  ...  momentum_kama  \
0            NaN    0.000000      

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


          Open      High       Low     Close  Volume  volume_adi  volume_obv  \
0          NaN       NaN       NaN       NaN   13884         NaN         NaN   
1    -0.023352 -0.013883 -0.005707 -0.012297   18075    2.376360   -1.301858   
2    -0.011002 -0.002211 -0.012032 -0.000055  168093    1.234050   40.108089   
3    -0.019716 -0.026922 -0.022347 -0.013277  224990   -3.029111    1.305925   
4    -0.033708 -0.000111 -0.017143 -0.018480   19715   -0.009672    0.049626   
...        ...       ...       ...       ...     ...         ...         ...   
4574  0.000000  0.000732  0.000000  0.002346  623895   -0.003712    0.001637   
4575  0.007743  0.004460  0.004613 -0.007753  707929    0.001992   -0.001855   
4576 -0.006952  0.004440  0.003185  0.015405  554084   -0.002798    0.001454   
4577  0.000000  0.000000  0.000000  0.000000  554148   -0.002807    0.001452   
4578  0.000000  0.000000  0.000000  0.000000  554148   -0.002814    0.001450   

      volume_cmf     volume_fi    volum

...

          Open      High       Low     Close   Volume  volume_adi  volume_obv  \
0          NaN       NaN       NaN       NaN  1268756         NaN         NaN   
1     0.037319  0.018393  0.037399  0.012805  1706608   -1.020647    1.345103   
2     0.000193 -0.002852  0.002575  0.004922   840360   -5.990568    0.282439   
3     0.008509 -0.003813 -0.021129 -0.021994   734988   -5.746625   -0.192621   
4    -0.023969  0.007464  0.000149  0.026318  2080960   -4.185961    0.675475   
...        ...       ...       ...       ...      ...         ...         ...   
3336 -0.008590 -0.020561 -0.013594 -0.007656  2088505   -0.006234   -0.002331   
3337 -0.004569  0.005388  0.008366  0.012466  1212566    0.039009    0.001357   
3338  0.009830  0.002401  0.007292 -0.001765  1274203   -0.016844   -0.001424   
3339 -0.005660  0.019589  0.000043  0.013671  3215610    0.024465    0.003598   
3340  0.021175  0.033644  0.025574  0.045437  6031344    0.291143    0.006725   

      volume_cmf     volume

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


          Open      High       Low     Close   Volume  volume_adi  volume_obv  \
0          NaN       NaN       NaN       NaN    80350         NaN         NaN   
1     0.003908  0.002806  0.002809  0.003159   849100    5.473768   10.567517   
2     0.009316  0.005430 -0.002241 -0.003010    73450    0.574821   -0.079025   
3    -0.008817 -0.008817 -0.006738 -0.010202   240320    1.996373   -0.280748   
4    -0.013481  0.019597  0.000933  0.016520   711310    0.169809    1.155324   
...        ...       ...       ...       ...      ...         ...         ...   
3336 -0.011817 -0.012288 -0.012945 -0.000165   548838   -0.004295   -0.001580   
3337  0.001480  0.008274  0.015498  0.014777   772158   -0.017061    0.002226   
3338  0.017329  0.004250  0.005157  0.004186  1025127   -0.022285    0.002949   
3339  0.004988  0.003105  0.006573 -0.002407   750126    0.007113   -0.002151   
3340 -0.007723  0.004088  0.000783  0.009180   832715   -0.016761    0.002393   

      volume_cmf     volume

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


          Open      High       Low     Close   Volume  volume_adi  volume_obv  \
0          NaN       NaN       NaN       NaN   952740         NaN         NaN   
1     0.010582  0.101937  0.012618  0.077535  7061550   11.284716    7.411833   
2     0.080628 -0.003754  0.055941  0.015643  2960980    0.523460    0.369463   
3     0.017345  0.018695  0.028680  0.003514  3807800   -0.355706    0.346944   
4    -0.002762  0.006320  0.007841  0.004988  2940330   -0.566743    0.198898   
...        ...       ...       ...       ...      ...         ...         ...   
3336 -0.004201 -0.012903 -0.011782 -0.009121  1177214    0.005631   -0.001002   
3337 -0.018419  0.001783  0.004562  0.017744  2167322   -0.011320    0.001846   
3338  0.022942  0.005694  0.015804  0.003511  1435731    0.000123    0.001221   
3339  0.002426  0.001415 -0.004292 -0.006997   985928    0.005219   -0.000837   
3340 -0.009504  0.004417 -0.001916  0.016899  1704921   -0.011339    0.001449   

      volume_cmf     volume

In [116]:

dataa = { "data":[]} 

for res in result:
    dataa["data"].append({ 
            "cdate": res[Icdate],
            "result": str(res[Iresult]),
            "pre_close" : str(res[Iclose]),            
            "isClassification": res[IisClassification],
            "model": res[Imodel],
            "stock": res[Istock],
            "prob": str(res[Iprob]),
    })
    
dataa

{'data': [{'cdate': '15-07-2023',
   'result': '0',
   'pre_close': '1373.6500244140625',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'OLECTRA',
   'prob': '51.47'},
  {'cdate': '15-07-2023',
   'result': '0',
   'pre_close': '688.7999877929688',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'CONCOR',
   'prob': '50.76'},
  {'cdate': '15-07-2023',
   'result': '0',
   'pre_close': '572.9500122070312',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'ELGIEQUIP',
   'prob': '54.26'},
  {'cdate': '15-07-2023',
   'result': '0',
   'pre_close': '96.5999984741211',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'IOC',
   'prob': '58.49'},
  {'cdate': '15-07-2023',
   'result': '0',
   'pre_close': '127.1500015258789',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'BEL',
   'prob': '55.74'},
  {'cdate': '15-07-2023',
   'result': '0',
   'pre_close': '7730.75',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'TATAELXSI',
   'prob': '50

In [117]:
import requests
import json

url = "https://website-development-kerala.com/api_214124524/ai_model_daily_runner.php"

response = requests.post(url, json=dataa)
print(response.status_code)
print(response)

200
<Response [200]>


In [118]:
final_target

{}

In [9]:
#for symbol in symbols:
   # retrained_model = loaded_models[symbol].fit(finaldata[symbol],final_target[symbol])
    #print(retrained_model)
    #with open('../TrainedModel/xg/{}_model_2.pkl'.format(symbol), 'wb') as f:
     #   pickle.dump(retrained_model, f)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=['error', 'logloss', 'logloss'], feature_types=None,
              gamma=6, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=25, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=1, ...)
XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eva